# A script to obtain the T3 solution

Action:
$$
S_{11} = \int d^{11}x \sqrt{-g} \left[R - \frac{1}{2}\frac{1}{4!}F_5^2\right]
$$

Metric ansatz:
$$
ds_{12}^2=H^a(-dx_0^2 + dx_1^2 + dx_2^2) + H^{-\frac{ad}{D-d-2}} (dy_0^2 + dy_1^2 + ... + dy_7^2)
$$

# 1. Compute Ricci tensor

parameters

In [1]:
D = 12
d = 4
HARMONIC_DIMENSIONS = D - d

set up coordinates

In [2]:
import sympy as sp
# coordinates
x = sp.symbols(f"x0:{d}", real=True)
y = sp.symbols(f"y0:{D - d}", real=True)
coords = (*x,*y)


# Parameters and symbols
apar = sp.symbols('a', real=True)
r_sym = sp.symbols('r', real=True, positive=True)

# expressions for radial coordinate r and r**2
r2 = sum(y_i**2 for y_i in y)
r = sp.sqrt(r2)

# harmonic function H, both as function of r, and as symbols
H = sp.Function('H')(r)
H_, H_p, H_pp = sp.symbols("H H' H''") # symbols for H and its derivatives, easier to work with once derivatives are taken and only manipulating algebraically

print(f"We are in {D} dimensions, with coordinates {coords}")
print(f"Radial coordinate r = {r}, r^2 = {r2}")
print(f"Harmonic function H is harmonic in {HARMONIC_DIMENSIONS} dimensions.")

We are in 12 dimensions, with coordinates (x0, x1, x2, x3, y0, y1, y2, y3, y4, y5, y6, y7)
Radial coordinate r = sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2 + y6**2 + y7**2), r^2 = y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2 + y6**2 + y7**2
Harmonic function H is harmonic in 8 dimensions.


In [3]:
from helpers.einstein_helpers import minkowski_metric, euclidean_metric
# metric
g_MN = sp.zeros(D)
g_MN[0:d,0:d] = H**(apar) * minkowski_metric(d)
g_MN[d:D,d:D] = H**(-apar * sp.Rational(d, D -d -2)) * euclidean_metric(D - d)
g_MN = sp.simplify(g_MN)
g = sp.simplify(sp.sqrt(-(sp.det(g_MN))))
inv_g_MN = g_MN.inv()

In [4]:
from helpers.einstein_helpers import ricci_tensor
R_MN = ricci_tensor(g_MN, coords)

Computed Gamma^0_...,Computed Gamma^1_...,Computed Gamma^2_...,Computed Gamma^3_...,Computed Gamma^4_...,Computed Gamma^5_...,Computed Gamma^6_...,Computed Gamma^7_...,Computed Gamma^8_...,Computed Gamma^9_...,Computed Gamma^10_...,Computed Gamma^11_...,Computed R[0,...],Computed R[1,...],Computed R[2,...],Computed R[3,...],Computed R[4,...],Computed R[5,...],Computed R[6,...],Computed R[7,...],Computed R[8,...],Computed R[9,...],Computed R[10,...],Computed R[11,...],

# 2. Simplifying relevant terms

From this point on no more derivatives shall be taken

In [5]:
def harmonic_derivative_subs(expr):
    # First sub in r and r^2
    expr = sp.simplify(expr)
    expr = sp.simplify(expr.subs({sp.sqrt(r2): r_sym,
                                  r2: r_sym**2}))
    
    # Then sub symbols of H, H', H'' for its literal derivatives
    expr = sp.simplify(expr.subs({H: H_,
                                  sp.Derivative(sp.Function('H')(r_sym), r_sym): H_p,
                                  sp.Derivative(sp.Function('H')(r_sym), (r_sym, 2)): H_pp}))
    
    # now impose harmonic condition to eliminate H''
    expr = expr.subs({H_pp: -(HARMONIC_DIMENSIONS -1)/r_sym * H_p})
    return sp.simplify(expr)

Obtain simplified expression for $R_{00}$

In [6]:
R_00 = harmonic_derivative_subs(R_MN[0,0]).subs(apar, sp.Rational(-3,5))
R_00 = sp.factor(R_00).subs(r**2, r_sym**2)
R_00

3*H'**2/(10*H(r)**3)

In [7]:
R_44 = harmonic_derivative_subs(R_MN[4,4]).subs(apar, sp.Rational(-3,5))
R_44 = R_44.subs(sum([2 * yi**2 for yi in y]), 2 * r_sym**2)
R_44

H'**2*(r**2 - 3*y0**2)/(5*r**2*H(r)**2)

# 3. form field

We are going to try

$$
(F_5)_{m 0 1 2 3} = c_1 \partial_m H^{d_1}
$$

and something else for its magnetic components which will involve $c_2, d_2$

In [8]:
c_1, c_2= sp.symbols('c_1 c_2')
d_1, d_2= sp.symbols('d_1 d_2', real=True)

d_1 = -sp.Rational(6,5)

In [9]:
from helpers.form_helpers import antisymmetrize_tensor
F5 = sp.MutableDenseNDimArray.zeros(*([D] * 5))

for m in range(8):
    # F[m+4,0,1,2,3] = c_1 * H**d_1 * sp.Derivative(H**(-1), y[m])
    F5[m+4,0,1,2,3] = c_1 * sp.Derivative(H**(d_1), y[m])

F5 = antisymmetrize_tensor(F5)

In [10]:
F7 = sp.MutableDenseNDimArray.zeros(*([D] * 7))

F7[4,5,6,7,8,9,10] = c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[11])
F7[5,6,7,8,9,10,11] = - c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[4])
F7[6,7,8,9,10,11,4] = c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[5])
F7[7,8,9,10,11,4,5] = - c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[6])
F7[8,9,10,11,4,5,6] = c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[7])
F7[9,10,11,4,5,6,7] = - c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[8])
F7[10,11,4,5,6,7,8] = c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[9])
F7[11,4,5,6,7,8,9] = - c_2 * H**d_2 * sp.Derivative(H**(d_2), coords[10])

F7 = antisymmetrize_tensor(F7)

In [11]:
# # this is speculative
# F[6,7,8,9,10] = c_2 * sp.Derivative(H**(d_2), coords[11])
# F[7,8,9,10,11] = -c_2 * sp.Derivative(H**(d_2), coords[6])
# F[8,9,10,11,6] = c_2 * sp.Derivative(H**(d_2), coords[7])
# F[9,10,11,6,7] = -c_2 * sp.Derivative(H**(d_2), coords[8])
# F[10,11,6,7,8] = c_2 * sp.Derivative(H**(d_2), coords[9])
# F[11,6,7,8,9] = -c_2 * sp.Derivative(H**(d_2), coords[10])
# F = antisymmetrize_tensor(F)

In [12]:
from helpers.form_helpers import compute_FF_MN, compute_S_MN, compute_form_squared
S5_MN = compute_S_MN(g_MN, 4, compute_FF_MN(F5, inv_g_MN), compute_form_squared(F5, inv_g_MN))
S7_MN = compute_S_MN(g_MN, 6, compute_FF_MN(F7, inv_g_MN), compute_form_squared(F7, inv_g_MN))

Computing F_M PQRS F_N^PQRS for a rank 5 form in 12 dimensions.
Computing squared norm of a rank 5 form in 12 dimensions.
Computing F_M PQRS F_N^PQRS for a rank 7 form in 12 dimensions.
Computing squared norm of a rank 7 form in 12 dimensions.


In [13]:
S_MN = S5_MN + S7_MN

In [14]:
S_00 = harmonic_derivative_subs(S_MN[0,0]).subs({apar: sp.Rational(-3,5),
                                                 d_1: sp.Rational(-6,5),
                                                 d_2: sp.Rational(3,5)})
S_00 = sp.factor(S_00).subs({r**2: r_sym**2,
                             sum([3 * yi**2 for yi in y]): 3 * r_sym**2,
                             sum([-7 * c_2**2 * yi**2 for yi in y]): -7 * c_2**2 * r_sym**2,
                             sum([12 * c_1**2 * yi**2 for yi in y]): 12 * c_1**2 * r_sym**2})
S_00

-9*H'**2*(-20*c_1**2*r**2 - 7*c_2**2*r**2)/(100*r**2*H(r)**3)

In [15]:
S_00 = sp.factor(S_00).subs({sum([3 * yi**2 for yi in y]): 3 * r_sym**2})
S_00

9*H'**2*(20*c_1**2 + 7*c_2**2)/(100*H(r)**3)

In [16]:
S_44 = harmonic_derivative_subs(S_MN[4,4]).subs({apar: sp.Rational(-3,5),
                                                 d_1: sp.Rational(-6,5),
                                                 d_2: sp.Rational(3,5)})
S_44 = sp.factor(S_44).subs({r**2: r_sym**2,
                             sum([-3 * yi**2 for yi in y[1:]]): -3 * (r_sym**2-y[0]**2),
                             sum([-5 * c_2**2 * yi**2 for yi in y[1:]]): -5 * c_2**2 * (r_sym**2-y[0]**2),
                             sum([-12 * c_1**2 * yi**2 for yi in y[1:]]): -12 * c_1**2 * (r_sym**2-y[0]**2),})
S_44

-9*H'**2*(20*c_1**2*y0**2 - 12*c_1**2*(r**2 - y0**2) + 7*c_2**2*y0**2 - 5*c_2**2*(r**2 - y0**2))/(100*r**2*H(r)**2)

# 4. trying to balance the equations

Trying to match the equations, we find $d_1 = -\frac{6}{5}, d_2 = -\frac{1}{5}$

In [17]:
eq00, eq44 = sp.Eq(R_00, S_00), sp.Eq(R_44, S_44)

In [18]:
sp.cancel(eq00)

Eq(3*H'**2/(10*H(r)**3), 9*H'**2*(20*c_1**2 + 7*c_2**2)/(100*H(r)**3))

In [19]:
eq44

Eq(H'**2*(r**2 - 3*y0**2)/(5*r**2*H(r)**2), -9*H'**2*(20*c_1**2*y0**2 - 12*c_1**2*(r**2 - y0**2) + 7*c_2**2*y0**2 - 5*c_2**2*(r**2 - y0**2))/(100*r**2*H(r)**2))